<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/Regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!pip install rawpy  ##tells collab that this is a cmd - whatever cmd , just prefix  !
import numpy as np 
import cv2 
import rawpy 
import imageio
from sklearn.linear_model import LinearRegression 
import os 
import math
import matplotlib.pyplot as plt 
import csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [5]:
from google.colab import drive 
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/RAW/'  #change dir to your project folder


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
##UPLOAD AIRVEDA CSV FIRST 
temp = os.listdir('/content/gdrive/My Drive/RAW/RAW_Batch1/')
jpg_names=[]
name_jpg=[]
dummy=[]
origin_time={}           #storing original timestamps 
#loading .jpg file names (ONLY Names) from RAW_Batch1
for name in temp: 
    path = '/content/gdrive/My Drive/RAW/RAW_Batch1/' + str(name)
    if(name.endswith('.jpg')):
      try: 
        name_jpg.append(path)
        dummy=name.split('_')
        k=str(dummy[1])+str(dummy[2])+str(dummy[3])+str(dummy[4])+str(dummy[5])
        jpg_names.append(k)
        origin_time[path]=k
      except: 
        print(name)


print(len(jpg_names))
print(len(origin_time))
print(len(name_jpg))


713
713
713


In [7]:
#Opening AirVeda csv 

temp={}
with open('/content/airveda.csv', mode ='r') as csv_file: 
      csv_reader=csv.reader(csv_file)
      for row in csv_reader: 
        temp[row[0]]=row[2]
stamp={}
for iter in temp: 
  try: 
    k=iter.split(':')
    stamp[(str(k[0])+ str(k[1]))]=temp[iter]
  except: 
    pass 
  
y={}
for stp in stamp: 
    x=stp.split('-')
    y[(str(x[0])+str(x[1])+str(x[2]))]=stamp[stp]

    
final_av={}
for ctr in y:
  z=ctr.split(' ')
  final_av[(str(z[0])+str(z[1]))]=y[ctr]
  

  
print(len(final_av))

2800


In [0]:
def transmission(img, lt = 230, retMean = False):
  ker = np.ones((3, 3))/9.0
  imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  b, g, r = cv2.split(img)
  hsv = cv2.cvtColor(imgrgb, cv2.COLOR_RGB2HSV)
  _, _, v = cv2.split(hsv)
  v_blur = cv2.filter2D(v, -1, ker)
  _,building_mask = cv2.threshold(v_blur, lt, 255, cv2.THRESH_BINARY)
  sky = cv2.bitwise_and(imgrgb, imgrgb, mask = building_mask)
  sky_mask = cv2.bitwise_not(building_mask, building_mask.copy())
  building = cv2.bitwise_and(imgrgb, imgrgb, mask = sky_mask)
  #f, ax = plt.subplots(2, 1, figsize = (15, 15))
  #ax[0].imshow(imgrgb)
  #ax[1].imshow(sky)
  #ax[2].imshow(building)
  al = max(v_blur.flatten())
  #Airlight Found. Now calculate Transmission Map
  img_norm = img/float(al)
  b, g, r = cv2.split(img_norm)
  kernel = np.ones((3,3),np.uint8)
  im_new = np.zeros((img.shape[0], img.shape[1]))
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      im_new[i][j] = min(b[i][j], g[i][j], r[i][j])
  dcp = cv2.erode(im_new, kernel, iterations = 1)
  #ax[3].imshow(im_new, cmap = 'gray')
  #ax[4].imshow(dcp, cmap = 'gray')
  tr = np.ones((dcp.shape[0], dcp.shape[1]))
  for i in range(dcp.shape[0]):
    for j in range(dcp.shape[1]):
      tr[i][j] = 1 - dcp[i][j]
  #ax[1].imshow(tr, cmap = 'gray')
  if retMean == False:
    return tr
  elif retMean == True:
    return np.mean(tr)

In [0]:
#Matching airveda data with jpeg names to getting matching timestamps 

match={}
for name in jpg_names:
  for stmp in final_av: 
    if(stmp==name and stmp not in match):
      match[stmp]=final_av[stmp]
      
#print((match))

      

timestamps=[]

for key in match:
  timestamps.append(str(key))
  
print(timestamps)
print(len(timestamps))

hours=[]
for stamp in timestamps: 
    hours.append(stamp[8:-2])

print(hours)
print(len(hours))
hours = np.array(hours, dtype=np.int32)

pm=[]
for p in match:
  pm.append(match[p])

print(pm)
print(len(pm))
pm=np.array(pm, dtype=np.int32)
#Plotting time against pm 

f,ax = plt.subplots(1,1,figsize=(10,10))
ax.scatter(hours,pm)


In [0]:
#Matching matched timestamps to the images that should be read 
crspond={}
for key in match: 
  for val in origin_time: 
    if(origin_time[val]==key):
      value=[]
      value.append(match[key])
      value.append(key)
      crspond[val]=value
      break
      

print(len(crspond))

trans_jpg=[]
entropy_jpg=[]
contrast_jpg=[]
images_jpg=[]
k=0
for key in crspond:
  try:
    read = cv2.imread(key)
    read = cv2.resize(read, (32, 32))
    images_jpg.append(read)
    k+=1
    print(k)
  except: 
    print(name)
    
    


In [0]:
print(crspond)

In [0]:
print(len(images_jpg))

In [0]:
print(images_jpg[-1])

In [0]:
def contrast(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  s = 0
  ss = 0
  mean = np.mean(image)
  x = (-1 * mean * (np.ones(image.shape)))
  ss = np.square(np.add(image, x))
  cont = np.sqrt(np.sum(ss)/float(image.size))
  return cont

In [0]:
def entropy(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = cv2.calcHist( [image.astype('float32')],
              [0], 
              None,
               [256], 
              [0,256] )
  h_norm = 0
  h_norm = cv2.normalize(hist, h_norm)
  ent = 0
  for p in h_norm:
    try:
      ent += p*math.log(p, 2)
    except:
      pass
  return (-1 * ent[0])

In [0]:
def transmission(img, lt = 230, retMean = False):
  ker = np.ones((3, 3))/9.0
  imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  b, g, r = cv2.split(img)
  hsv = cv2.cvtColor(imgrgb, cv2.COLOR_RGB2HSV)
  _, _, v = cv2.split(hsv)
  v_blur = cv2.filter2D(v, -1, ker)
  _,building_mask = cv2.threshold(v_blur, lt, 255, cv2.THRESH_BINARY)
  sky = cv2.bitwise_and(imgrgb, imgrgb, mask = building_mask)
  sky_mask = cv2.bitwise_not(building_mask, building_mask.copy())
  building = cv2.bitwise_and(imgrgb, imgrgb, mask = sky_mask)
  #f, ax = plt.subplots(2, 1, figsize = (15, 15))
  #ax[0].imshow(imgrgb)
  #ax[1].imshow(sky)
  #ax[2].imshow(building)
  al = max(v_blur.flatten())
  #Airlight Found. Now calculate Transmission Map
  img_norm = img/float(al)
  b, g, r = cv2.split(img_norm)
  kernel = np.ones((3,3),np.uint8)
  im_new = np.zeros((img.shape[0], img.shape[1]))
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      im_new[i][j] = min(b[i][j], g[i][j], r[i][j])
  dcp = cv2.erode(im_new, kernel, iterations = 1)
  #ax[3].imshow(im_new, cmap = 'gray')
  #ax[4].imshow(dcp, cmap = 'gray')
  tr = np.ones((dcp.shape[0], dcp.shape[1]))
  for i in range(dcp.shape[0]):
    for j in range(dcp.shape[1]):
      tr[i][j] = 1 - dcp[i][j]
  #ax[1].imshow(tr, cmap = 'gray')
  if retMean == False:
    return tr
  elif retMean == True:
    return np.mean(tr)

In [0]:
trans_jpg=[]
for img in images_jpg: 
  k=transmission(img, retMean= True)
  trans_jpg.append(k)
  print(k)

print(len(trans_jpg))



In [0]:
contrast_jpg=[]
for img in images_jpg: 
  k=contrast(img)
  print(k)
  contrast_jpg.append(k)

In [0]:
entropy_jpg=[]
for img in images_jpg: 
  k=entropy(img)
  print(k)
  entropy_jpg.append(k)

In [0]:
!pip install urlopen
import json 
import requests 
import urlopen
from pandas.io.json import json_normalize

with open('/content/weather_historic.json') as file: 
  data=json.load(file)
  store=data["data"]
  table=json_normalize(store['weather'])

  #print(json.dumps(data, indent =4))  
  index=table.index
  columns=table.columns 
  values=table.values

 

In [0]:
#!pip install pickle 
import pickle 
'''([int(year+month+day1+time[0:2]+"00"),float(hour["windspeedKmph"]),float(hour["tempC"]),float(hour["precipMM"]), float(hour["HeatIndexC"]), float(hour["humidity"]), float(hour["pressure"]), float(hour["visibility"]) ])'''

with open('/content/data (2).bin', 'rb') as file:
  atad = pickle.load(file)

# print(len(atad))
# print(atad)

#print(len(data))

windspeed=[]
temp=[]
prec=[]

humidity=[]
pressure=[]
visibility=[]


for e in atad:
  for j in e:
    windspeed.append(j[0])
    temp.append(j[1])
    prec.append(j[2])
    humidity.append(j[4])
    pressure.append(j[5])
    visibility.append(j[6])

    
# print(windspeed)
# print(visibility)


In [0]:
print(len(timestamps))
print(len(trans_jpg))
print(len(entropy_jpg))
print(len(contrast_jpg))
print(len(windspeed))
print(len(temp))
print(len(humidity))
print(len(pressure))
print(len(visibility))
print(len(prec))
#Mapping final timestamps to weather data 





In [59]:
#FEATURE FINETUNING 

#Converting timestamps to int 
hours=np.reshape(np.array(list(map(int,hours))), (322, 1))
transmission=np.reshape(np.array(list(map(float,trans_jpg))), (322, 1))
entropy=np.reshape(np.array(list(map(float,entropy_jpg))), (322, 1))
contrast=np.reshape(np.array(list(map(float,contrast_jpg))),(322,1))
windspeed=np.reshape(windspeed, (322,1))
temp=np.reshape(temp, (322,1))
humidity=np.reshape(humidity, (322,1))
pressure=np.reshape(pressure, (322,1))
visibility=np.reshape(visibility, (322,1))
prec=np.reshape(prec, (322,1))

features=np.hstack((hours, transmission, entropy, contrast))
print(features)



[[18.          0.73241419 44.10022736 68.74766409]
 [18.          0.72683441 44.27140045 69.35324794]
 [18.          0.7314876  46.13936615 68.04237884]
 ...
 [18.          0.74400659 44.17639923 70.76689819]
 [18.          0.7405216  45.16226578 70.11360843]
 [18.          0.72589614 46.42108536 68.82514667]]


In [65]:
#REGRESSION 
regr = LinearRegression()
labels = pm
from sklearn.model_selection import train_test_split
features = np.hstack((hours,transmission,entropy, contrast, windspeed, temp, humidity, pressure, visibility, prec))
print(np.shape(labels))
print(np.shape(features))
x_train, x_test, y_train, y_test = train_test_split(features,labels,test_size=0.3, random_state= 42 )
regr.fit(x_train, y_train)  
pred = regr.predict(x_test)
print(pred)

print(mean_squared_error(pred, y_test)**0.5)



(322,)
(322, 10)
[40.78434478 40.00096658 39.03803042 46.94796187 33.49010404 43.88497733
 52.78259263 47.8153511  46.19340392 78.24437598 39.29011659 40.5736488
 69.85598861 39.97177991 41.23192675 64.61496359 40.53296595 41.62908549
 64.57969068 58.17937595 37.43113406 40.76615756 73.90731016 72.03390236
 78.49294839 39.81375915 42.53129718 39.3619013  40.10380995 52.37362871
 34.5364134  34.97788604 39.91178092 36.88972761 51.43215265 41.22930562
 77.04030972 69.47284135 45.94031086 35.65967315 76.59818891 68.07225959
 70.87383386 28.09826052 75.03190765 76.45184443 41.00533309 40.53275714
 46.43944629 40.34983466 40.44227581 57.13198202 41.58491353 41.45217823
 68.75766471 66.00239255 52.0905055  35.1420519  45.96795301 41.89860702
 43.20244894 39.66692291 46.01728168 56.29243502 41.3980355  41.71630575
 44.34520794 65.09492935 40.7109347  38.5425685  40.83454733 36.84855672
 42.00427011 53.49099997 71.51506491 38.21425217 37.55634688 29.58894566
 47.19765414 71.2505326  64.0269647